## Using Wannier Function to construct Hamiltonian
While it is possible to build the models by writting out all the terms, it is difficult if we need to build a accurate model or if the system is complicated. Thanks to the wannier function method, we can extract a tight binding Hamiltonian from DFT data. Minimulti is interfaced withe Wannier90 so it can read the output from Wannier90 and build models on top of that.  

In this example, a model for SrMnO3 is built. We first calcualte the Wannier function Hamiltonian from DFT+Wannier90 in a non-magnetic system. We use non-spin-polarized GGA in the DFT calculation. Then in the model Hamiltonian  a Hubbard correction is added to the tight binding Hamiltonian, which enables the simulation of magnetic properties.

Here the O2p and Mn 3d orbitals are considered in the Hamiltonian. The result below shows that three Mn t2g obitals in the same spin channle are occupied and the other 3d states are empty.

In [1]:
%matplotlib inline
from minimulti.electron.Hamiltonian import load_pickle_to_model 
import numpy as np
import matplotlib.pyplot as plt
import os

def run_SrMnO3_FM(sc_matrix=[[1, 0, 0], [0, 1, 0], [0, 0, 1]],# supercell matrix
                U=7, # U value
                J=1, # J value
                Utype='Kanamori',
                spinat=[0.0] * 9 + [0,1,1,0,1],  # spinat for each orb. [pz, px, py]*3 , [dz2, dxy, dyz, dx^2-y^2, dxz]
):
    model_fname='data/SrMnO3_wannier/SrMnO3.pickle'
    #if not os.path.exists(output_directory):
    #    os.makedirs(output_directory)
    spinat = np.array(spinat)
    model = load_pickle_to_model(model_fname)
    model.set_Hubbard_U(
        Utype=Utype,
        #Utype='Kanamori',
        Hubbard_dict={'Mn': {
            'U': U,
            'J': J,
        }})
    model.set_kmesh([6, 6, 6])
    model.set_initial_spin(spinat)
    model.set(nel=21, tol_energy=1e-6, tol_rho=1e-6, mixing=0.5, sigma=0.1)
    model.scf_solve(print_iter_info=False)
    #etotal=model._total_energy
    #print(model.get_site_charges())
    #print(model.get_site_magnetic_moments())
    kvectors = [[0, 0, 0], [0, 0.5, 0], [0.5, 0.5, 0], [0, 0, 0],
                [0.5, 0.5, 0.5]]
    
    # plot band structure with projections to orbitals
    model.plot_band(
        supercell_matrix=sc_matrix,
        kvectors=kvectors,
        knames=['$\Gamma$', 'X', 'M', "$\Gamma$", 'R']
        )
    plt.show()

    #charges = model.get_site_charges()
    #spins = model.get_site_magnetic_moments()

In [2]:
%matplotlib inline
from ipywidgets import FloatSlider, interact, interact_manual, fixed
#run_SrMnO3_FM( U=8, J=1)
interact_manual(run_SrMnO3_FM,
                sc_matrix=fixed([[1, 0, 0], [0, 1, 0], [0, 0, 1]]),
                U=(0, 8,0.5),
                J=(0,2, 0.1), 
                Utype=['Kanamori', 'Liechtenstein'], 
                spinat=fixed([0.0] * 9 + [0,1,1,0,1]));

interactive(children=(FloatSlider(value=7.0, description='U', max=8.0, step=0.5), FloatSlider(value=1.0, descr…